##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 卷积神经网络（Convolutional Neural Network, CNN）

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/tutorials/images/cnn"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 tensorFlow.google.cn 上查看</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/images/cnn.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a>   </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/images/cnn.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a>   </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tutorials/images/cnn.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载 notebook</a>   </td>
</table>

本教程展示了如何训练一个简单的[卷积神经网络](https://developers.google.com/machine-learning/glossary/#convolutional_neural_network) (CNN) 来对 [CIFAR 图像](https://www.cs.toronto.edu/~kriz/cifar.html)进行分类。由于本教程使用的是 [Keras Sequential API](https://tensorflow.google.cn/guide/keras/overview)，创建和训练模型只需要几行代码。


### 导入 TensorFlow

In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

### 下载并准备 CIFAR10 数据集

CIFAR10 数据集包含 10 类，共 60000 张彩色图片，每类图片有 6000 张。此数据集中 50000 个样例被作为训练集，剩余 10000 个样例作为测试集。类之间相互独立，不存在重叠的部分。

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

### 验证数据

为了验证数据集看起来是否正确，我们绘制训练集中的前 25 张图像并在每张图像下方显示类名称：


In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

### 构造卷积神经网络模型

下方展示的 6 行代码声明了了一个常见卷积神经网络，由几个 [Conv2D](https://tensorflow.google.cn/api_docs/python/tf/keras/layers/Conv2D) 和 [MaxPooling2D](https://tensorflow.google.cn/api_docs/python/tf/keras/layers/MaxPool2D) 层组成。

CNN 将形状为 (image_height, image_width, color_channels) 的张量作为输入，忽略批次大小。如果您不熟悉这些维度，color_channels 是指 (R,G,B)。在此示例中，您将配置 CNN 以处理形状为 (32, 32, 3) 的输入，即 CIFAR 图像的格式。您可以通过将参数 `input_shape` 传递给第一层来实现此目的。


In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

到目前为止，模型的架构如下：

In [ ]:
model.summary() 

在上面的结构中，您可以看到每个 Conv2D 和 MaxPooling2D 层的输出都是一个三维的张量 (Tensor)，其形状描述了 (height, width, channels)。越深的层中，宽度和高度都会收缩。每个 Conv2D 层输出的通道数量 (channels) 取决于声明层时的第一个参数（如：上面代码中的 32 或 64）。这样，由于宽度和高度的收缩，您便可以（从运算的角度）增加每个 Conv2D 层输出的通道数量 (channels)。

### 增加 Dense 层

为了完成模型，您需要将卷积基（形状为 (4, 4, 64)）的最后一个输出张量馈送到一个或多个 Dense 层以执行分类。Dense 层将向量作为输入（即 1 维），而当前输出为 3 维张量。首先，将 3 维输出展平（或展开）为 1 维，然后在顶部添加一个或多个 Dense 层。CIFAR 有 10 个输出类，因此使用具有 10 个输出的最终 Dense 层。

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

下面是模型的完整架构：

In [ ]:
model.summary()

网络摘要显示 (4, 4, 64) 输出在经过两个 Dense 层之前被展平为形状为 (1024) 的向量。

### 编译并训练模型

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

### 评估模型

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.show()

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
print(test_acc)

您的简单 CNN 的测试准确率已达到 70% 以上。对于只有几行的代码来说，效果不错！对于另一种 CNN 风格，请参阅[适合专家的 TensorFlow 2 快速入门](https://tensorflow.google.cn/tutorials/quickstart/advanced)示例，此示例使用了 Keras 子类化 API 和 `tf.GradientTape`。